<a href="https://colab.research.google.com/github/ramonVDAKKER/teaching/blob/main/QF_illustration_solving_black_scholes_pde.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantitative Finance - solving the Black-Scholes Partial Differential Equation numerically

We seek the function $V=V(t,s)$ satisfying the Black-Scholes PDE
$$
\frac{\delta V}{\delta t}(t,s) + r s \frac{\partial V}{\partial
s}(t,s) + \frac{1}{2}\sigma^2 s^2 \frac{\partial^2 V}{\partial
s^2}(t,s) -r V(t,s)=0\quad \forall s\in [0,\infty),\quad \forall t\in [0,T],
$$
subject to the constraint
$V(T,s)=F_T(s)$ $\forall s\in [0,\infty)$,
where $F_T$, the terminal boundary condition, is the payoff at maturity of the option whose price we want to determine (for example, $F_T(s)=\max\{K-s,0\}$ for a European put option with strike price $K$ and maturity $T$).



Sometimes it is possible to find an analytical solution to the PDE (e.g. the Black-Scholes formula for the price of a European call option).
How does this work? The Black-Scholes PDE can be transformed in the heat equation. The heat equation is a PDE that is relevant in physics and because of that a well-studied object in mathematics and physics. So if you need to price a derivative with payoff $F_T$ you can try to rewrite the problem in terms of the heat equation and to consult the literature on the heat equation whether an analytical solution is known for this boundary condition.
In general, one will resort to numerical techniques to determine the solution to the Black-Scholes PDE. Below we discuss one of the simplest numerical algorithms.


Divide the time-interval $[0, T]$ into $N$ equally sized subintervals of length $dt$. The price of the underlying asset will in principle take values in $[0,\infty)$. In the algorithm an artificial limit,  $S_{\text{max}}$ is introduced. The size of
$S_{\text{max}}$ requires experimentation. It is not hard to imagine that this choice should be related to the shape of $s\mapsto F_T(s)$.
Next, the interval $[0, S_{\text{max}}]$ is divided into $M$ equally sized subintervals of length $ds$.
So we are going to approximate the continuous space $[0, T]\times [0,\infty)$ by a finite grid $(t_i, s_j )$, where $t_i = i\cdot dt$ and
$s_j = j\cdot ds$, $i\in\{0, 1, . . . ,N\}$ and $j\in\{0, 1, . . . ,M\}$. In the following we abbreviate $V(t_i,s_j)$ to $V_{i,j}$.
Next we use the following numerical approximations to the derivatives,
$$\frac{\partial V}{\partial s}(t_i,s_j)\approx \frac{ V_{i,j+1} -V_{i,j-1}} {2 ds }, $$
$$
\frac{\partial^2 V}{\partial s^2}(t_i,s_j)\approx\frac{ V_{i,j+1} -2V_{i,j} +V_{i,j-1}} { (ds)^2 },
$$
$$
\frac{\partial V}{\partial t}(t_i,s_j)\approx \frac{ V_{i+1,j} -V_{i,j}} {dt }.
$$


Inserting these approximations into the Black-Scholes PDE we arrive at
\begin{align*}
\frac{ V_{i+1,j} -V_{i,j}} {dt } + rjds \frac{ V_{i,j+1} -V_{i,j-1}} {2 ds }  + \frac{1}{2}\sigma^2 (jds)^2
\frac{ V_{i,j+1} -2V_{i,j} +V_{i,j-1}} { (ds)^2 }
-r V_{i,j}=0.
\end{align*}
Introducing
\begin{align*}
a_j &= \frac{1}{2}rjdt - \frac{1}{2}\sigma^2j^2dt, \\
b_j &= 1 + \sigma^2j^2 dt + rdt, \\
c &= -\frac{1}{2}rjdt - \frac{1}{2}\sigma^2 j^2dt,
\end{align*}
we can rewrite the equation as follows:
\begin{align*}
a_j V_{i,j-1}+b_j V_{i,j}+c_j V_{i,j+1}- V_{i+1,j}=0.
\end{align*}
Fixing $i\in\{0,\dots,N-1\}$ and reformulating the equations of the previous display into matrix notation we obtain
\begin{align*}
\begin{pmatrix}
b_1 & c_1 & 0   & 0   & 0   & \cdots   & 0 \\
a_2 & b_2 & c_2 & 0   & 0   & \cdots   & 0 \\
0   & a_3 & b_3 & c_3 & 0   & \cdots   & 0 \\
\vdots & \ddots & \ddots & \ddots & \ddots & \ddots & \vdots \\
\vdots & \vdots & \ddots & \ddots & \ddots & \ddots & \vdots \\
0      &   0    &0 &0 & a_{M-2} & b_{M-2} & c_{M-2}  \\
0 & 0 & 0 & 0 & 0 & a_{M-1} & b_{M-1}
\end{pmatrix}
\begin{pmatrix}
V_{i,1} \\ \vdots \\ \vdots \\ \vdots \\ \vdots \\ V_{i,M-1}
\end{pmatrix}
=
\begin{pmatrix}
V_{i+1,1}-a_1 V_{i,0}
\\
V_{i+1,2}
\\
\\ \vdots \\ \vdots \\  V_{i+1,M-2} \\ V_{i+1,M-1}-c_{M-1} V_{i+1,M}.
\end{pmatrix}
\end{align*}
The boundary values $V_{i,0}$ and $V_{i,M}$ should be derived by ad hoc arguments and are specific for the derivative of interest.
For a European put option, $F_T(s)=\max\{K-s,0\}$, we can take $V_{i,0}=K$ and $V_{i,M}=0$.

Below we consider a basic implementation in Python.

## 0. Import packages

In [ ]:
!pip install  scipy==1.7.1 # we need recent version

In [ ]:
import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 1. Example

Consider the following parameters for the Black-Scholes market and the put:

In [ ]:
s_0 = 100
K = 90
sigma = .15
r = .01
T = 3

The following function can be used to determine the numerical approximation to the put price. The algorithm requires a choice for the tuning parameters Smax, dS, and dT. Besides a (visual) presentation of the put prices, the function also returns the price of the put at t=0 using $S_0$=s_0. If s_0 is not in the grid, the point that is closest to s_0 is used.

In [ ]:
def approximation_price_put(Smax: float, dS: float, K: float, dT:float, T: float, r: float, sigma: float):
  """This function implements the algorithm that has been described above."""

  if K > Smax:
    raise ValueError("Smax should be larger than the strike price.")
  # preparations in order to define grid:
  M = np.int(np.ceil(Smax / dS)) # number of points in grid for stockprice
  ds = Smax / M  # mesh in grid for stockprice
  N = int(np.ceil(T / dT)) # number of points in grid for time
  dt = T / N # mesh in grid for time
  # define time grid and grid for stock price:
  t = np.linspace(0, T, N + 1)
  S = np.linspace(0, Smax, M + 1)
  # intialize value function V (i.e. price option on grid):
  V = np.zeros((N + 1, M + 1))  # time x stock price
  # set boundary conditions (for payoff at maturity):
  V[N, :] = np.maximum(K - S, 0) # boundary at t=T, i.e. pay-off
  V[:, M] = 0 # boundary a t  and S=Smax, note that this approximation only makes sense if Smax is large enough!
  V[:, 0] = K # if S = 0 then V = K for all t 
  # set up difference equation
  J = np.arange(1, M - 1 + 1)
  a = .5 * r  * J * dt - 0.5 * sigma ** 2  * J ** 2 * dt
  b = 1 + sigma ** 2 * dt * J ** 2 + r * dt
  c = -0.5 * r * dt * J - .5 * sigma ** 2 * dt *J ** 2
  A = diags([a[1:], b, c[: -1]], offsets=[-1, 0, 1])
  # solve V recursively
  for i in range(N, 0, -1):
    y = np.ravel(V[i, 1 : M])
    y[0] = y[0] - a[0] * K
    V[i - 1, 1 : M] = np.transpose(spsolve(A, y))
  fig = plt.figure(figsize=(15,15))
  ax = plt.axes(projection='3d')
  stockprice, time = np.meshgrid(S, t)
  ax.plot_surface(time, stockprice, V,  cmap="gray")
  ax.set_xlabel("time")
  ax.set_ylabel("stockprice")
  ax.set_zlabel("put price")  
  ax.view_init(20, 0)
  return V, S, t

In [ ]:
# Grid settings
Smax = 250
dS = .32
dT = 1 / 250
V, S, _ = approximation_price_put(Smax, dS, K, dT, T, r, sigma)

In [ ]:
nearest_idx = np.where(abs(S - s_0) == abs(S - s_0).min())[0][0] # find index of point in stock price grid closest to S_0
S_0_proxy = S[nearest_idx] # If you directly want the element of array (array) nearest to the given number (num)
print(f"The closest point to S_0 on the grid is {S_0_proxy}")
print(f"The (approximation to the) price of the put at t=0, S_0=s_0 is {V[0, nearest_idx]}")